In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
## Make Cost Function
from sklearn.metrics import make_scorer
import math

/home/jiashen/env3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
def get_feature_importance(model):
    Importance = model.get_fscore()
    Importance = list(Importance.items())
    Feature= []
    Score = []
    for each in Importance:
        Feature.append(each[0])
        Score.append(each[1])
    df = pd.DataFrame({'Feature':Feature,'Score':Score}).sort_values(by=['Score'],ascending=[0])
    return df    

In [2]:
def rmsle(preds, dtrain):
    labels = dtrain.get_label()
    assert len(preds) == len(labels)
    labels = labels.tolist()
    preds = preds.tolist()
    terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0, preds[i]) + 1)) ** 2.0 for i, pred in enumerate(labels)]
    return 'rmsle', (sum(terms_to_sum) * (1.0 / len(preds))) ** 0.5

In [3]:
train = pd.read_csv('train_all_features.csv')
test = pd.read_csv('test_all_features.csv')

In [4]:
test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,x_school_km_theater_km,x_public_healthcare_km_green_zone_km,x_public_healthcare_km_workplaces_km,x_public_healthcare_km_cemetery_km,x_public_healthcare_km_theater_km,x_radiation_km_cemetery_km,x_tsne_2_cemetery_km,x_green_zone_km_cemetery_km,x_green_zone_km_theater_km,x_workplaces_km_theater_km
0,30474,2015-07-01,39.0,20.7,2,9.0,1,1998.0,1.0,8.9,...,0.003578,0.000635,0.003015,0.001161,0.004775,0.006781,0.000015,0.001667,0.006853,0.032537
1,30475,2015-07-01,79.2,NaN,8,17.0,1,NaN,3.0,NaN,...,0.004151,0.000000,0.008656,0.007851,0.007303,0.025161,0.095754,0.000000,0.000000,0.025833
2,30476,2015-07-01,40.5,25.1,3,5.0,2,1960.0,2.0,4.8,...,0.000136,0.002617,0.000582,0.002200,0.000304,0.003666,0.201835,0.068319,0.009439,0.002100
3,30477,2015-07-01,62.8,36.0,17,17.0,1,2016.0,2.0,NaN,...,0.015393,0.000939,0.015379,0.014362,0.015495,0.033432,0.019466,0.002386,0.002575,0.042152
4,30478,2015-07-01,40.0,40.0,17,17.0,1,NaN,1.0,NaN,...,0.003980,0.005625,0.004599,0.004266,0.004098,0.026670,0.152824,0.033369,0.032056,0.026205


In [5]:
col = list(test.columns)[2:]

In [7]:
label = train['price_doc']

In [8]:
dtrain = xgb.DMatrix(train[col],label)
dtest = xgb.DMatrix(test[col])

In [11]:
params = {
    'eta': 0.05, ## Try 0.01,3,5
    'max_depth': 7,## Try 4,5,6
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    #'eval_metric': rmsle,
    'silent': 1
    #'lambda':5,
    #'min_child_weight':5
}

In [12]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=20,feval=rmsle,
    verbose_eval=50, show_stdv=False,seed=42)
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmsle:2.96492	test-rmsle:2.97034
[50]	train-rmsle:0.424123	test-rmsle:0.456963
Performance does not improve from 54 rounds


In [13]:
model = xgb.train(params,dtrain,num_boost_round=54)

In [14]:
pred = model.predict(dtest)

In [16]:
sub = pd.DataFrame({'id':test['id'],'price_doc':pred})

In [17]:
sub.to_csv('RMSLEWithAdditionalFeatures.csv',index=False)

In [19]:
get_feature_importance(model)

,Feature,Score
79,full_sq,366
123,life_sq,104
321,x_full_sq_extra_area,87
32,x_full_sq_micex_cbi_tr,76
212,x_full_sq_tsne_2,53
67,floor,49
189,build_year,47
327,cafe_count_5000_price_2500,46
307,metro_min_avto,45
160,x_floor_railroad_km,45
